<a href="https://colab.research.google.com/github/jallenrobern/CCPGLANG/blob/main/PAN_Syntax_Analyzer_COOK%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install flask flask-cors pyngrok

In [14]:
from pyngrok import ngrok

# 🔐 Set your auth token
ngrok.set_auth_token("3497TDD7p62qutWxLVvQHGq3iTo_2PF64hjRKWCwwhNGEyfpK")

In [15]:
# ==================== LEXER ==================== v3
import re
from enum import Enum
from dataclasses import dataclass

class TokenType(Enum):
    # Keywords / Commands
    ADD = 'ADD'
    MIX = 'MIX'
    STIR = 'STIR'
    CHOP = 'CHOP'
    FRY = 'FRY'
    BOIL = 'BOIL'
    BAKE = 'BAKE'
    SERVE = 'SERVE'
    WAIT = 'WAIT'
    SET = 'SET'
    LET = 'LET'
    IF = 'IF'
    ELSE = 'ELSE'
    REPEAT = 'REPEAT'
    TIMES = 'TIMES'
    PROCEDURE = 'PROCEDURE'
    RETURN = 'RETURN'
    PRINT = 'PRINT'
    WHILE = 'WHILE'

    # Operators / Symbols
    ASSIGN = 'ASSIGN'      # =
    PLUS = 'PLUS'          # +
    MINUS = 'MINUS'        # -
    MULT = 'MULT'          # *
    DIV = 'DIV'            # /
    EQ = 'EQ'              # ==
    NE = 'NE'              # !=
    LT = 'LT'              # <
    GT = 'GT'              # >
    LE = 'LE'              # <=
    GE = 'GE'              # >=
    LPAREN = 'LPAREN'      # (
    RPAREN = 'RPAREN'      # )
    LBRACE = 'LBRACE'      # {
    RBRACE = 'RBRACE'      # }
    SEMICOLON = 'SEMICOLON'
    COMMA = 'COMMA'

    # Data types
    ID = 'ID'
    NUMBER = 'NUMBER'
    STRING = 'STRING'

    EOF = 'EOF'
    ERROR = 'ERROR'


@dataclass
class Token:
    type: TokenType
    value: any
    line: int
    col: int


class Lexer:
    def __init__(self, code):
        self.code = code
        self.tokens = []
        self.line = 1
        self.col = 1
        self.keywords = {
            "add": TokenType.ADD,
            "mix": TokenType.MIX,
            "stir": TokenType.STIR,
            "chop": TokenType.CHOP,
            "fry": TokenType.FRY,
            "boil": TokenType.BOIL,
            "bake": TokenType.BAKE,
            "serve": TokenType.SERVE,
            "wait": TokenType.WAIT,
            "set": TokenType.SET,
            "let": TokenType.LET,
            "if": TokenType.IF,
            "else": TokenType.ELSE,
            "repeat": TokenType.REPEAT,
            "times": TokenType.TIMES,
            "procedure": TokenType.PROCEDURE,
            "return": TokenType.RETURN,
            "print": TokenType.PRINT,
            "while": TokenType.WHILE,
        }

    def tokenize(self):
        code = self.code
        token_spec = [
            ("NUMBER",   r'\d+(\.\d+)?'),
            ("STRING",   r'"[^"\n]*"'),
            ("ID",       r'[A-Za-z_][A-Za-z0-9_]*'),
            ("EQ",       r'=='),
            ("NE",       r'!='),
            ("LE",       r'<='),
            ("GE",       r'>='),
            ("ASSIGN",   r'='),
            ("LT",       r'<'),
            ("GT",       r'>'),
            ("PLUS",     r'\+'),
            ("MINUS",    r'-'),
            ("MULT",     r'\*'),
            ("DIV",      r'/'),
            ("LPAREN",   r'\('),
            ("RPAREN",   r'\)'),
            ("LBRACE",   r'\{'),
            ("RBRACE",   r'\}'),
            ("SEMICOLON",r';'),
            ("COMMA",    r','),
            ("NEWLINE",  r'\n'),
            ("SKIP",     r'[ \t]+'),
            ("MISMATCH", r'.'),
        ]

        tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_spec)

        for mo in re.finditer(tok_regex, code):
            kind = mo.lastgroup
            value = mo.group()
            if kind == "NUMBER":
                tok = Token(TokenType.NUMBER, float(value) if '.' in value else int(value), self.line, self.col)
            elif kind == "STRING":
                tok = Token(TokenType.STRING, value.strip('"'), self.line, self.col)
            elif kind == "ID":
                tok_type = self.keywords.get(value.lower(), TokenType.ID)
                tok = Token(tok_type, value, self.line, self.col)
            elif kind == "NEWLINE":
                self.line += 1
                self.col = 1
                continue
            elif kind == "SKIP":
                self.col += len(value)
                continue
            elif kind == "MISMATCH":
                tok = Token(TokenType.ERROR, value, self.line, self.col)
            else:
                tok = Token(TokenType[kind], value, self.line, self.col)

            self.tokens.append(tok)
            self.col += len(value)

        self.tokens.append(Token(TokenType.EOF, None, self.line, self.col))
        return self.tokens

# ==================== PARSER ====================

class ParseNode:
    def __init__(self, node_type: str, children=None, token=None):
        self.type = node_type
        self.children = children or []
        self.token = token

    def to_dict(self):
        return {
            'type': self.type,
            'value': self.token.value if self.token else None,
            'children': [child.to_dict() for child in self.children]
        }

    def to_string(self, depth=0):
        indent = "  " * depth
        result = f"{indent}{self.type}"
        if self.token:
            result += f" [{self.token.value}]"
        result += "\n"
        for child in self.children:
            result += child.to_string(depth + 1)
        return result

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        self.errors = []   # <-- collect errors here

    # === Utility ===
    def current_token(self):
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        return Token(TokenType.EOF, None, -1, -1)

    def advance(self):
        if self.pos < len(self.tokens) - 1:
            self.pos += 1

    def expect(self, token_type):
        if self.current_token().type == token_type:
            return True
        self.error(f"Expected {token_type.name} but found {self.current_token().type.name}")
        return False

    def error(self, message):
        token = self.current_token()
        err = {'line': token.line, 'column': token.col, 'message': message}
        self.errors.append(err)
        # keep printing for debugging in server logs
        print(f"[Syntax Error] Line {token.line}:{token.col} → {message}")

    # === Entry point ===
    def parse(self):
        node = ParseNode("program")
        while self.current_token().type != TokenType.EOF:
            node.children.append(self.parse_statement())
        return node

    # === Statement parser ===
    def parse_statement(self):
        token = self.current_token()

        if token.type == TokenType.PROCEDURE:
            return self.parse_procedure_def()
        elif token.type == TokenType.REPEAT:
            return self.parse_repeat_loop()
        elif token.type == TokenType.WHILE:
            return self.parse_while_loop()
        elif token.type == TokenType.IF:
            return self.parse_if_stmt()
        elif token.type == TokenType.LET:
            return self.parse_declaration()
        elif token.type == TokenType.SET:
            return self.parse_assignment()
        elif token.type in {
            TokenType.CHOP, TokenType.MIX, TokenType.STIR, TokenType.FRY,
            TokenType.BOIL, TokenType.BAKE, TokenType.SERVE, TokenType.WAIT
        }:
            return self.parse_cook_command()
        elif token.type == TokenType.PRINT:
            return self.parse_print_stmt()
        elif token.type == TokenType.RETURN:
            node = ParseNode("return_stmt")
            self.advance()
            if not self.expect(TokenType.SEMICOLON):
                return node
            self.advance()
            return node
        elif token.type == TokenType.ID:
            # Could be a procedure call
            return self.parse_procedure_call()
        else:
            self.error(f"Unexpected token {token.type.name}")
            self.advance()
            return ParseNode("error")

    # === Declarations ===
    def parse_declaration(self):
        node = ParseNode("declaration")
        self.advance()  # skip 'let'

        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("id", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected variable name after 'let'")
            return node

        if self.current_token().type == TokenType.ASSIGN:
            self.advance()
            node.children.append(self.parse_expr())
        else:
            self.error("Expected '=' in declaration")

        if not self.expect(TokenType.SEMICOLON):
            return node
        self.advance()
        return node

    # === Assignment ===
    def parse_assignment(self):
        node = ParseNode("assignment")
        self.advance()  # skip 'set'

        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("id", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected variable name after 'set'")
            return node

        if not self.expect(TokenType.ASSIGN):
            return node
        self.advance()

        node.children.append(self.parse_expr())

        if not self.expect(TokenType.SEMICOLON):
            return node
        self.advance()
        return node

    # === While loop ===
    def parse_while_loop(self):
        node = ParseNode("while_loop")
        self.advance()  # skip 'while'

        if not self.expect(TokenType.LPAREN):
            return node
        self.advance()

        node.children.append(self.parse_condition())

        if not self.expect(TokenType.RPAREN):
            return node
        self.advance()

        if not self.expect(TokenType.LBRACE):
            return node
        self.advance()

        node.children.append(self.parse_statement_list())

        if not self.expect(TokenType.RBRACE):
            return node
        self.advance()

        return node

    # === Repeat loop ===
    def parse_repeat_loop(self):
        node = ParseNode("repeat_loop")
        self.advance()  # skip 'repeat'

        if self.current_token().type == TokenType.NUMBER:
            node.children.append(ParseNode("number", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected number after 'repeat'")

        if not self.expect(TokenType.TIMES):
            return node
        self.advance()

        if not self.expect(TokenType.LBRACE):
            return node
        self.advance()

        node.children.append(self.parse_statement_list())

        if not self.expect(TokenType.RBRACE):
            return node
        self.advance()

        return node

    # === If–Else ===
    def parse_if_stmt(self):
        node = ParseNode("if_stmt")
        self.advance()  # skip 'if'

        if not self.expect(TokenType.LPAREN):
            return node
        self.advance()

        node.children.append(self.parse_condition())

        if not self.expect(TokenType.RPAREN):
            return node
        self.advance()

        if not self.expect(TokenType.LBRACE):
            return node
        self.advance()

        node.children.append(self.parse_statement_list())

        if not self.expect(TokenType.RBRACE):
            return node
        self.advance()

        if self.current_token().type == TokenType.ELSE:
            self.advance()
            if not self.expect(TokenType.LBRACE):
                return node
            self.advance()
            node.children.append(ParseNode("else_block", children=[self.parse_statement_list()]))
            if not self.expect(TokenType.RBRACE):
                return node
            self.advance()

        return node

    # === Procedures ===
    def parse_procedure_def(self):
        node = ParseNode("procedure_def")
        self.advance()  # skip 'procedure'

        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("id", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected procedure name after 'procedure'")
            return node

        if not self.expect(TokenType.LPAREN):
            return node
        self.advance()

        # parameters (comma-separated ids)
        params = ParseNode("params")
        while self.current_token().type == TokenType.ID:
            params.children.append(ParseNode("param", token=self.current_token()))
            self.advance()
            if self.current_token().type == TokenType.COMMA:
                self.advance()
            else:
                break
        node.children.append(params)

        if not self.expect(TokenType.RPAREN):
            return node
        self.advance()

        if not self.expect(TokenType.LBRACE):
            return node
        self.advance()

        node.children.append(self.parse_statement_list())

        if not self.expect(TokenType.RBRACE):
            return node
        self.advance()

        return node

    # === Procedure call ===
    def parse_procedure_call(self):
        node = ParseNode("procedure_call")
        node.children.append(ParseNode("id", token=self.current_token()))
        self.advance()

        if not self.expect(TokenType.LPAREN):
            return node
        self.advance()

        args = ParseNode("args")
        if self.current_token().type != TokenType.RPAREN:
            args.children.append(self.parse_expr())
            while self.current_token().type == TokenType.COMMA:
                self.advance()
                args.children.append(self.parse_expr())

        node.children.append(args)

        if not self.expect(TokenType.RPAREN):
            return node
        self.advance()

        if not self.expect(TokenType.SEMICOLON):
            return node
        self.advance()

        return node

    # === Cooking commands ===
    def parse_cook_command(self):
        node = ParseNode("cook_command", token=self.current_token())
        self.advance()

        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("ingredient", token=self.current_token()))
            self.advance()

        if self.current_token().type == TokenType.NUMBER:
            node.children.append(ParseNode("time", token=self.current_token()))
            self.advance()

        if not self.expect(TokenType.SEMICOLON):
            return node
        self.advance()
        return node

    # === Print statement ===
    def parse_print_stmt(self):
        node = ParseNode("print_stmt")
        self.advance()

        if not self.expect(TokenType.LPAREN):
            return node
        self.advance()

        node.children.append(self.parse_expr())

        if not self.expect(TokenType.RPAREN):
            return node
        self.advance()

        if not self.expect(TokenType.SEMICOLON):
            return node
        self.advance()

        return node

    # === Statement list (for blocks) ===
    def parse_statement_list(self):
        node = ParseNode("statement_list")
        while self.current_token().type not in {TokenType.RBRACE, TokenType.EOF}:
            node.children.append(self.parse_statement())
        return node

    # === Expressions and conditions ===
    def parse_condition(self):
        node = ParseNode("condition")
        node.children.append(self.parse_expr())

        if self.current_token().type in {TokenType.EQ, TokenType.NE, TokenType.LT,
                                         TokenType.GT, TokenType.LE, TokenType.GE}:
            node.children.append(ParseNode("operator", token=self.current_token()))
            self.advance()
            node.children.append(self.parse_expr())

        return node

    def parse_expr(self):
        node = self.parse_term()
        while self.current_token().type in {TokenType.PLUS, TokenType.MINUS}:
            op = ParseNode("operator", token=self.current_token())
            self.advance()
            right = self.parse_term()
            node = ParseNode("binop", children=[node, op, right])
        return node

    def parse_term(self):
        node = self.parse_factor()
        while self.current_token().type in {TokenType.MULT, TokenType.DIV}:
            op = ParseNode("operator", token=self.current_token())
            self.advance()
            right = self.parse_factor()
            node = ParseNode("binop", children=[node, op, right])
        return node

    def parse_factor(self):
        token = self.current_token()

        if token.type == TokenType.NUMBER:
            node = ParseNode("number", token=token)
            self.advance()
            return node

        elif token.type == TokenType.STRING:
            node = ParseNode("string", token=token)
            self.advance()
            return node

        elif token.type == TokenType.ID:
            node = ParseNode("id", token=token)
            self.advance()
            return node

        elif token.type == TokenType.LPAREN:
            self.advance()
            node = self.parse_expr()
            if not self.expect(TokenType.RPAREN):
                return node
            self.advance()
            return node

        else:
            self.error(f"Unexpected token {token.type.name}")
            self.advance()
            return ParseNode("error")

# ==================== FLASK APP ====================
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

@app.route('/')
def index():
    html = '''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>PAN Analyzer - Google Colab</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 20px; }
        .container { max-width: 1400px; margin: 0 auto; background: white; border-radius: 15px; box-shadow: 0 20px 60px rgba(0,0,0,0.3); overflow: hidden; display: flex; flex-direction: column; min-height: 90vh; }
        .header { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; text-align: center; }
        .header h1 { font-size: 2.5em; margin-bottom: 10px; }
        .main-content { display: flex; flex: 1; gap: 20px; padding: 20px; }
        .editor-section, .output-section { display: flex; flex-direction: column; flex: 1; background: #f8f9fa; border-radius: 10px; border: 2px solid #e9ecef; }
        .section-title { background: #667eea; color: white; padding: 15px 20px; font-weight: bold; }
        .editor-wrapper { display: flex; flex: 1; position: relative; }
        .line-numbers { background: #2d2d2d; color: #858585; padding: 15px 10px; font-family: 'Courier New', monospace; font-size: 14px; text-align: right; user-select: none; min-width: 50px; border-right: 2px solid #667eea; overflow: hidden; line-height: 1.5; white-space: pre; }
        textarea { flex: 1; padding: 15px; border: none; font-family: 'Courier New', monospace; font-size: 14px; resize: none; line-height: 1.5; overflow-y: auto; }
        textarea:focus { outline: none; background: #f0f4ff; }
        .controls { display: flex; gap: 10px; padding: 15px; background: white; border-top: 2px solid #e9ecef; }
        button { padding: 12px 25px; font-size: 1em; border: none; border-radius: 8px; cursor: pointer; font-weight: bold; transition: all 0.3s; }
        .btn-analyze { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; flex: 1; }
        .btn-analyze:hover { box-shadow: 0 10px 20px rgba(102,126,234,0.4); transform: translateY(-2px); }
        .btn-clear { background: #e9ecef; }
        .btn-clear:hover { background: #dee2e6; }
        .btn-example { background: #28a745; color: white; }
        .btn-example:hover { background: #218838; box-shadow: 0 5px 15px rgba(40,167,69,0.3); }
        .output-content { flex: 1; overflow-y: auto; padding: 20px; background: white; }
        .message { margin-bottom: 15px; padding: 15px; border-radius: 8px; border-left: 5px solid; font-family: monospace; font-size: 12px; }
        .success { background: #d4edda; border-color: #28a745; color: #155724; }
        .error { background: #f8d7da; border-color: #dc3545; color: #721c24; }
        .status-badge { display: inline-block; padding: 8px 15px; border-radius: 20px; font-weight: bold; margin-bottom: 15px; }
        .status-success { background: #d4edda; color: #155724; }
        .status-error { background: #f8d7da; color: #721c24; }
        .section-subtitle { font-weight: bold; color: #667eea; margin-top: 15px; margin-bottom: 8px; }
        .token-list { background: #f8f9fa; padding: 10px; border-radius: 5px; margin-bottom: 15px; font-size: 12px; max-height: 250px; overflow-y: auto; }
        .token-item { padding: 5px 8px; margin: 3px 0; background: white; border-left: 3px solid #667eea; border-radius: 3px; }
        .tree-container { background: #f8f9fa; padding: 10px; border-radius: 5px; font-size: 11px; max-height: 300px; overflow-y: auto; white-space: pre-wrap; word-break: break-all; font-family: 'Courier New', monospace; }
        @media (max-width: 1024px) { .main-content { flex-direction: column; } }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🍳 PAN Syntax Analyzer</h1>
            <p>Programming Language for Cooking - Supports Tagalog identifiers!</p>
        </div>

        <div class="main-content">
            <div class="editor-section">
                <div class="section-title">📝 Code Editor</div>
                <div class="editor-wrapper">
                    <div class="line-numbers" id="lineNumbers">1</div>
                    <textarea id="codeInput" spellcheck="false">procedure makeAdobo() {
    add manok;
    add suka;
    boil 30;
    return;
}

makeAdobo();</textarea>
                </div>
                <div class="controls">
                    <button class="btn-analyze" onclick="analyzeCode()">▶ Analyze Code</button>
                    <button class="btn-clear" onclick="clearCode()">🗑 Clear</button>
                    <button class="btn-example" onclick="loadExample()">📚 Example</button>
                </div>
            </div>

            <div class="output-section">
                <div class="section-title">📊 Analysis Results</div>
                <div class="output-content" id="outputContent">
                    <p style="color: #999; text-align: center; margin-top: 50px;">👈 Click "Analyze Code" to see results</p>
                </div>
            </div>
        </div>
    </div>

    <script>
        const codeInput = document.getElementById('codeInput');
        const lineNumbers = document.getElementById('lineNumbers');

        function updateLineNumbers() {
            const lines = codeInput.value.split('\\n');
            const numbers = lines.map((_, i) => i + 1).join('\\n');
            lineNumbers.textContent = numbers;
        }

        codeInput.addEventListener('input', updateLineNumbers);
        codeInput.addEventListener('scroll', () => {
            lineNumbers.scrollTop = codeInput.scrollTop;
        });

        updateLineNumbers();

        async function analyzeCode() {
            const code = codeInput.value;
            const output = document.getElementById('outputContent');

            if (!code.trim()) {
                output.innerHTML = '<div class="message error">⚠️ Please enter code</div>';
                return;
            }

            output.innerHTML = '<p style="color: #999; text-align: center; margin-top: 50px;">⏳ Analyzing...</p>';

            try {
                const response = await fetch('/analyze', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({code: code})
                });

                const data = await response.json();

                let html = '';
                const hasErrors = data.errors.length > 0;
                const statusClass = hasErrors ? 'status-error' : 'status-success';
                const statusText = hasErrors ? '❌ ERRORS FOUND' : '✅ SYNTAX VALID';
                html += `<div class="status-badge ${statusClass}">${statusText}</div>`;

                if (hasErrors) {
                    html += `<div class="message error"><strong>Errors: ${data.errors.length}</strong></div>`;
                    data.errors.forEach(err => {
                        html += `<div class="message error">🔴 Line ${err.line}, Col ${err.column}: ${err.message}</div>`;
                    });
                } else {
                    html += `<div class="message success"><strong>✓ Syntax valid! Walang errors.</strong></div>`;
                }

                html += '<div class="section-subtitle">📋 Tokens</div><div class="token-list">';
                data.tokens.forEach(t => {
                    html += `<div class="token-item"><strong>${t.type}</strong> → ${t.value} (L${t.line}:C${t.col})</div>`;
                });
                html += '</div>';

                html += '<div class="section-subtitle">🌳 Parse Tree</div><div class="tree-container">' + escapeHtml(data.parse_tree) + '</div>';

                output.innerHTML = html;
            } catch(e) {
                output.innerHTML = `<div class="message error">❌ Error: ${e.message}</div>`;
            }
        }

        function clearCode() {
            codeInput.value = '';
            updateLineNumbers();
            document.getElementById('outputContent').innerHTML = '<p style="color: #999; text-align: center; margin-top: 50px;">👈 Click "Analyze Code" to see results</p>';
        }

        function loadExample() {
            const examples = [
                `procedure makeAdobo() {
    add manok;
    add suka;
    add bawang;
    boil 30;
    return;
}

makeAdobo();`,
                `set temperatura = 350;
if (temperatura > 300) {
    print("Init na ang apoy");
} else {
    print("Dagdagan ang init");
}
fry kanin 5;
mix;`,
                `set bilang = 0;
repeat 3 times {
    stir;
    set bilang = bilang + 1;
}
add itlog;
bake 15;
serve;`,
                `procedure lutongUlam(sangkap) {
    chop sangkap;
    fry sangkap 10;
    mix;
    return;
}

lutongUlam(gulay);
serve;`
            ];
            codeInput.value = examples[Math.floor(Math.random() * examples.length)];
            updateLineNumbers();
            setTimeout(analyzeCode, 100);
        }

        function escapeHtml(text) {
            const map = {'&': '&amp;', '<': '&lt;', '>': '&gt;', '"': '&quot;', "'": '&#039;'};
            return text.replace(/[&<>"']/g, m => map[m]);
        }
    </script>
</body>
</html>'''
    return html

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.json
    code = data.get('code', '')

    try:
        lexer = Lexer(code)
        tokens = lexer.tokenize()

        parser = Parser(tokens)
        tree = parser.parse()

        return jsonify({
            'tokens': [{'type': t.type.value, 'value': str(t.value), 'line': t.line, 'col': t.col} for t in tokens[:-1]],
            'errors': parser.errors,
            'parse_tree': tree.to_string(),
            'status': 'valid' if not parser.errors else 'error'
        })
    except Exception as e:
        # Catch unexpected exceptions and return JSON (avoid sending HTML traceback)
        import traceback
        tb = traceback.format_exc()
        print("Internal server error during analysis:", tb)  # server log
        return jsonify({
            'tokens': [],
            'errors': [{'line': -1, 'column': -1, 'message': f'Internal error: {str(e)}'}],
            'parse_tree': '',
            'status': 'error'
        }), 500

# ==================== RUN SERVER ====================

# Kill any existing process on port 5000
import os

# Create ngrok tunnel
try:
    ngrok.kill()
except:
    pass

# Set up ngrok with auth token
ngrok.set_auth_token("3497TDD7p62qutWxLVvQHGq3iTo_2PF64hjRKWCwwhNGEyfpK")

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f"\n{'='*60}")
print(f"✅ PAN ANALYZER IS LIVE!")
print(f"{'='*60}")
print(f"🌐 PUBLIC URL: {public_url}")
print(f"{'='*60}")
print(f"\n📝 Instructions:")
print(f"1. Click the link above or copy it to your browser")
print(f"2. Write your PAN code in the editor (supports Tagalog!)")
print(f"3. Click 'Analyze Code' to see results")
print(f"4. View errors, tokens, and parse tree")
print(f"\n💡 Tagalog examples:")
print(f"   - Variables: manok, suka, bawang, gulay, temperatura")
print(f"   - Identifiers: makeAdobo, lutongUlam, sangkap")
print(f"\n⏱️  Keep this cell running while using the analyzer!")
print(f"   (The link will work for 8 hours)\n")

# Run the Flask app
app.run(port=5000, debug=False)


✅ PAN ANALYZER IS LIVE!
🌐 PUBLIC URL: NgrokTunnel: "https://mycelial-ronan-vagabondish.ngrok-free.dev" -> "http://localhost:5000"

📝 Instructions:
1. Click the link above or copy it to your browser
2. Write your PAN code in the editor (supports Tagalog!)
3. Click 'Analyze Code' to see results
4. View errors, tokens, and parse tree

💡 Tagalog examples:
   - Variables: manok, suka, bawang, gulay, temperatura
   - Identifiers: makeAdobo, lutongUlam, sangkap

⏱️  Keep this cell running while using the analyzer!
   (The link will work for 8 hours)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Oct/2025 13:24:14] "POST /analyze HTTP/1.1" 200 -
